Imports 

In [ ]:
from pyscope.reduction import avg_fits, ccd_calib

from astropy.io import fits
import matplotlib as mpl
import matplotlib.pyplot as plt
import smplotlib

import logging

# # Set up logging
# logging.basicConfig(level=logging.DEBUG)

# # Set up after cell logging at debug level
# logger = logging.getLogger('pyscope')
# logger.setLevel(logging.DEBUG)

mpl.rcParams['figure.dpi'] = 300

Create master dark

In [ ]:
avg_fits(
    fnames="./images/calibrations/2024-05-03T17-56-00/dark_read2_gain4x_1x1/*.fts",
    outfile="./images/calibrations/2024-05-03T17-56-00/master_dark_read2_gain4x_1x1.fts",
    datatype="float32",
)

Plot master dark

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
data = fits.getdata("./images/calibrations/2024-05-03T17-56-00/master_dark_read2_gain4x_1x1.fts")
mu = data.mean()
sigma = data.std()
im_plt = ax.imshow(data, cmap="gray", origin="lower", vmin=mu-3*sigma, vmax=mu+3*sigma)
cbar = plt.colorbar(im_plt, ax=ax)
ax.set_title("Master Dark")
plt.show()

Create master bias

In [ ]:
avg_fits(
    fnames="./images/calibrations/2024-05-03T17-56-00/bias_read2_gain4x_1x1/*.fts",
    outfile="./images/calibrations/2024-05-03T17-56-00/master_bias_read2_gain4x_1x1.fts",
)

Plot master bias

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
data = fits.getdata("./images/calibrations/2024-05-03T17-56-00/master_bias_read2_gain4x_1x1.fts")
mu = data.mean()
std = data.std()
im_plt = ax.imshow(data, cmap="gray", origin="lower", vmin=mu - 3 * std, vmax=mu + 3 * std)
cbar = plt.colorbar(im_plt, ax=ax)
ax.set_title("Master Bias")
plt.show()

Calibrate flats

In [ ]:
filt = "r"

ccd_calib(
    fnames=f"./images/calibrations/2024-05-03T17-56-00/flat_{filt}_read2_gain4x_1x1/*.fts",
    dark_frame="./images/calibrations/2024-05-03T17-56-00/master_dark_read2_gain4x_1x1.fts",
    bias_frame="./images/calibrations/2024-05-03T17-56-00/master_bias_read2_gain4x_1x1.fts",
    # in_place=True,
    #verbose=2
)

Plot one of the calibrated flats

In [ ]:
fname = "r.flat1_cal.fts"

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
data = fits.getdata(f"./images/calibrations/2024-05-03T17-56-00/flat_{filt}_read2_gain4x_1x1/{fname}")
mu = data.mean()
sigma = data.std()
im_plot = ax.imshow(data, cmap='gray', vmin=mu - 3 * sigma, vmax=mu + 3 * sigma)
cbar = fig.colorbar(im_plot, ax=ax)
ax.set_title('Calibrated flat')
plt.show()

Make master flat

In [ ]:
avg_fits(
    fnames=f"./images/calibrations/2024-05-03T17-56-00/flat_{filt}_read2_gain4x_1x1/*.fts",
    outfile=f"./images/calibrations/2024-05-03T17-56-00/master_flat_{filt}_read2_gain4x_1x1.fts",
    pre_normalize=True,
)

Plot master flat

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
data = fits.getdata(f"./images/calibrations/2024-05-03T17-56-00/master_flat_{filt}_read2_gain4x_1x1.fts")
mu = data.mean()
sigma = data.std()
im_plot = ax.imshow(data, cmap='gray', vmin=mu - 3 * sigma, vmax=mu + 3 * sigma)
cbar = fig.colorbar(im_plot, ax=ax)
ax.set_title('Master flat')
plt.show()

Calibrate a science image

In [ ]:
science_fname = "D:\RLMT\images\mjc_mjcMay02_TIC 48217457_r_90s_2024-05-03T06-59-30_wcs.fts"

ccd_calib(
    science_fname,
    dark_frame="./images/calibrations/2024-05-03T17-56-00/master_dark_read2_gain4x_1x1.fts",
    bias_frame="./images/calibrations/2024-05-03T17-56-00/master_bias_read2_gain4x_1x1.fts",
    flat_frame=f"./images/calibrations/2024-05-03T17-56-00/master_flat_{filt}_read2_gain4x_1x1.fts",
)

Plot the calibrated science image

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
data = fits.getdata(science_fname.split(".")[0] + "_cal.fts")
hdr = fits.getheader(science_fname.split(".")[0] + "_cal.fts")
if 'PEDESTAL' in hdr:
    data = data - hdr['PEDESTAL']
mu = data.mean()
sigma = data.std()
im_plot = ax.imshow(data, cmap='gray', vmin=mu - 0.5 * sigma, vmax=mu + 1.5 * sigma)
cbar = fig.colorbar(im_plot, ax=ax)
ax.set_title('Calibrated science frame')
plt.show()